**BERT LARGE on SQuAD 1.1**

Based on the code provided at https://github.com/google-research/bert

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import csv

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime' 
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  print(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

In [0]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']
  

!test -d sq_repo || git clone https://github.com/rajpurkar/SQuAD-explorer sq_repo
if not 'sq_repo' in sys.path:
  sys.path += ['sq_repo']

In [0]:
# SQuaD 1.1
TRAIN_FILE = "sq_repo/dataset/train-v1.1.json"
EVAL_FILE = "sq_repo/dataset/dev-v1.1.json"

In [0]:
TASK = "SQuAD_1.1_large_new_1"
#BERT_MODEL = 'wwm_uncased_L-24_H-1024_A-16' #@param {type:"string"}
BERT_MODEL = 'uncased_L-24_H-1024_A-16' #@param {type:"string"}
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL
#BERT_PRETRAINED_DIR = 'gs://bert_bucket_new/bert/Whole Word Masking'
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR
#!gsutil ls $'gs://bert_bucket_new/bert/Whole Word Masking'

BUCKET = 'bert_bucket_new' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-24_H-1024_A-16/vocab.txt
***** Model output directory: gs://bert_bucket_new/bert/models/SQuAD_1.1_large_new_1 *****


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import json
import math
import os
import random
import modeling
import optimization
import tokenization
import six
import tensorflow as tf
import numpy as np

WITH_NEGATIVE = False
NULL_SCORE_DIFF_THRESHOLD = 0.0
TRAIN_BATCH_SIZE = 16#24
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 3e-5#3e-5
NUM_TRAIN_EPOCHS = 2.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 386#389#386#384

# Model configs
VERBOSE_LOGGING = False
SAVE_CHECKPOINTS_STEPS = 500
ITERATIONS_PER_LOOP = 500
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')
DOC_STRIDE = 128

In [0]:
class SquadExample(object):
  """A single training/test example for simple sequence classification.

     For examples without an answer, the start and end position are -1.
  """

  def __init__(self,
               qas_id,
               question_text,
               doc_tokens,
               orig_answer_text=None,
               char_to_word_offset=None,
               start_position=None,
               end_position=None,
               is_impossible=False):
    self.qas_id = qas_id
    self.question_text = question_text
    self.doc_tokens = doc_tokens
    self.orig_answer_text = orig_answer_text
    self.start_position = start_position
    self.end_position = end_position
    self.is_impossible = is_impossible
    self.char_to_word_offset = char_to_word_offset

  def __str__(self):
    return self.__repr__()

  def __repr__(self):
    s = ""
    s += "qas_id: %s" % (tokenization.printable_text(self.qas_id))
    s += ", question_text: %s" % (
        tokenization.printable_text(self.question_text))
    s += ", doc_tokens: [%s]" % (" ".join(self.doc_tokens))
    if self.start_position:
      s += ", start_position: %d" % (self.start_position)
    if self.start_position:
      s += ", end_position: %d" % (self.end_position)
    if self.start_position:
      s += ", is_impossible: %r" % (self.is_impossible)
    return s


class InputFeatures(object):
  """A single set of features of data."""

  def __init__(self,
               unique_id,
               example_index,
               doc_span_index,
               tokens,
               token_to_orig_map,
               token_is_max_context,
               input_ids,
               input_mask,
               segment_ids,
               start_position=None,
               end_position=None,
               is_impossible=None):
    self.unique_id = unique_id
    self.example_index = example_index
    self.doc_span_index = doc_span_index
    self.tokens = tokens
    self.token_to_orig_map = token_to_orig_map
    self.token_is_max_context = token_is_max_context
    self.input_ids = input_ids
    self.input_mask = input_mask
    self.segment_ids = segment_ids
    self.start_position = start_position
    self.end_position = end_position
    self.is_impossible = is_impossible


def read_squad_examples(input_file, is_training):
  """Read a SQuAD json file into a list of SquadExample."""
  with tf.gfile.Open(input_file, "r") as reader:
    input_data = json.load(reader)["data"] ##########

  def is_whitespace(c):
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
      return True
    return False

  examples = []
  for entry in input_data:
    for paragraph in entry["paragraphs"]:
      paragraph_text = paragraph["context"]
      doc_tokens = []
      char_to_word_offset = []
      prev_is_whitespace = True
      for c in paragraph_text:
        if is_whitespace(c):
          prev_is_whitespace = True
        else:
          if prev_is_whitespace:
            doc_tokens.append(c)
          else:
            doc_tokens[-1] += c
          prev_is_whitespace = False
        char_to_word_offset.append(len(doc_tokens) - 1)
      for qa in paragraph["qas"]:
        qas_id = qa["id"]
        question_text = qa["question"]
        start_position = None
        end_position = None
        orig_answer_text = None
        is_impossible = False
        if is_training:

          if WITH_NEGATIVE:
            is_impossible = qa["is_impossible"]
          if (len(qa["answers"]) != 1) and (not is_impossible):
            raise ValueError(
                "For training, each question should have exactly 1 answer.")
          if not is_impossible:
            answer = qa["answers"][0]
            orig_answer_text = answer["text"]
            answer_offset = answer["answer_start"]
            answer_length = len(orig_answer_text)
            start_position = char_to_word_offset[answer_offset]
            end_position = char_to_word_offset[answer_offset + answer_length -
                                               1]
            # Only add answers where the text can be exactly recovered from the
            # document. If this CAN'T happen it's likely due to weird Unicode
            # stuff so we will just skip the example.
            #
            # Note that this means for training mode, every example is NOT
            # guaranteed to be preserved.
            
            actual_text = " ".join(
                doc_tokens[start_position:(end_position + 1)])
            cleaned_answer_text = " ".join(
                tokenization.whitespace_tokenize(orig_answer_text))
            if actual_text.find(cleaned_answer_text) == -1:
              tf.logging.warning("Could not find answer: '%s' vs. '%s'",
                                 actual_text, cleaned_answer_text)
              continue
          else:
            start_position = -1
            end_position = -1
            orig_answer_text = ""

        example = SquadExample(
            qas_id=qas_id,
            question_text=question_text,
            doc_tokens=doc_tokens,
            orig_answer_text=orig_answer_text,
            char_to_word_offset=char_to_word_offset,
            start_position=start_position,
            end_position=end_position,
            is_impossible=is_impossible)
        examples.append(example)

  return examples


def convert_examples_to_features(examples, tokenizer, max_seq_length,
                                 doc_stride, max_query_length, is_training,
                                 output_fn):
  """Loads a data file into a list of `InputBatch`s."""

  all_tokens = []
  unique_id = 1000000000

  all_features_matrix = []
  for i in range(2):
  
    all_features_list = []
    for (example_index, example) in enumerate(examples[i]):
      query_tokens = tokenizer.tokenize(example.question_text)

      if len(query_tokens) > max_query_length:
        query_tokens = query_tokens[0:max_query_length]

      tok_to_orig_index = []
      orig_to_tok_index = []
      all_doc_tokens = []
      for (i, token) in enumerate(example.doc_tokens):
        orig_to_tok_index.append(len(all_doc_tokens))
        sub_tokens = tokenizer.tokenize(token)
        for sub_token in sub_tokens:
          tok_to_orig_index.append(i)
          all_doc_tokens.append(sub_token)

      tok_start_position = None
      tok_end_position = None
      if is_training and example.is_impossible:
        tok_start_position = -1
        tok_end_position = -1
      if is_training and not example.is_impossible:
        tok_start_position = orig_to_tok_index[example.start_position]
        if example.end_position < len(example.doc_tokens) - 1:
          tok_end_position = orig_to_tok_index[example.end_position + 1] - 1
        else:
          tok_end_position = len(all_doc_tokens) - 1
        (tok_start_position, tok_end_position) = _improve_answer_span(
            all_doc_tokens, tok_start_position, tok_end_position, tokenizer,
            example.orig_answer_text)

      # The -3 accounts for [CLS], [SEP] and [SEP]
      max_tokens_for_doc = max_seq_length - len(query_tokens) - 3

      # We can have documents that are longer than the maximum sequence length.
      # To deal with this we do a sliding window approach, where we take chunks
      # of the up to our max length with a stride of `doc_stride`.
      _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
          "DocSpan", ["start", "length"])
      doc_spans = []
      start_offset = 0
      while start_offset < len(all_doc_tokens):
        length = len(all_doc_tokens) - start_offset
        if length > max_tokens_for_doc:
          length = max_tokens_for_doc
        doc_spans.append(_DocSpan(start=start_offset, length=length))
        if start_offset + length == len(all_doc_tokens):
          break
        start_offset += min(length, doc_stride)

      for (doc_span_index, doc_span) in enumerate(doc_spans):
        tokens = []
        token_to_orig_map = {}
        token_is_max_context = {}
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in query_tokens:
          tokens.append(token)
          segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)

        for i in range(doc_span.length):
          split_token_index = doc_span.start + i
          token_to_orig_map[len(tokens)] = tok_to_orig_index[split_token_index]

          is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                                 split_token_index)
          token_is_max_context[len(tokens)] = is_max_context
          tokens.append(all_doc_tokens[split_token_index])
          segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
          input_ids.append(0)
          input_mask.append(0)
          segment_ids.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length 

        start_position = None
        end_position = None
        if is_training and not example.is_impossible:
          # For training, if our document chunk does not contain an annotation
          # we throw it out, since there is nothing to predict.
          doc_start = doc_span.start
          doc_end = doc_span.start + doc_span.length - 1
          out_of_span = False
          if not (tok_start_position >= doc_start and
                  tok_end_position <= doc_end):
            out_of_span = True
          if out_of_span:
            start_position = 0
            end_position = 0
          else:
            doc_offset = len(query_tokens) + 2
            start_position = tok_start_position - doc_start + doc_offset
            end_position = tok_end_position - doc_start + doc_offset

        if is_training and example.is_impossible:
          start_position = 0
          end_position = 0

        if example_index < 20:
          tf.logging.info("*** Example ***")
          tf.logging.info("unique_id: %s" % (unique_id))
          tf.logging.info("example_index: %s" % (example_index))
          tf.logging.info("doc_span_index: %s" % (doc_span_index))
          tf.logging.info("tokens: %s" % " ".join(
              [tokenization.printable_text(x) for x in tokens]))
          tf.logging.info("token_to_orig_map: %s" % " ".join(
              ["%d:%d" % (x, y) for (x, y) in six.iteritems(token_to_orig_map)]))
          tf.logging.info("token_is_max_context: %s" % " ".join([
              "%d:%s" % (x, y) for (x, y) in six.iteritems(token_is_max_context)
          ]))
          tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
          tf.logging.info(
              "input_mask: %s" % " ".join([str(x) for x in input_mask]))
          tf.logging.info(
              "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
          if is_training and example.is_impossible:
            tf.logging.info("impossible example")
          if is_training and not example.is_impossible:
            answer_text = " ".join(tokens[start_position:(end_position + 1)])
            tf.logging.info("start_position: %d" % (start_position))
            tf.logging.info("end_position: %d" % (end_position))
            tf.logging.info(
                "answer: %s" % (tokenization.printable_text(answer_text)))

        feature = InputFeatures(
            unique_id=unique_id,
            example_index=example_index,
            doc_span_index=doc_span_index,
            tokens=tokens,
            token_to_orig_map=token_to_orig_map,
            token_is_max_context=token_is_max_context,
            input_ids=input_ids,
            input_mask=input_mask,
            segment_ids=segment_ids,
            start_position=start_position,
            end_position=end_position,
            is_impossible=example.is_impossible)

        # Run callback
        all_features_list.append(feature)
        #output_fn(feature)     
      unique_id += 1
      all_tokens.append(tokens)
      
    all_features_matrix.append(all_features_list)
  all_features_matrix = np.array(all_features_matrix)
  
  print(len(all_features_matrix[0]))
  print(len(all_features_matrix[1]))
  
  assert len(all_features_matrix[0]) == len(all_features_matrix[1])
  
  for i in range(len(all_features_matrix[1])):
    f = [all_features_matrix[0][i], all_features_matrix[1][i]]#, all_features_matrix[2][i]]
    output_fn(f)
    #print(f)

  print("Processed All Features!!!!!")
  return all_tokens, all_features_matrix
      
def _check_is_max_context(doc_spans, cur_span_index, position):
  """Check if this is the 'max context' doc span for the token."""

  # Because of the sliding window approach taken to scoring documents, a single
  # token can appear in multiple documents. E.g.
  #  Doc: the man went to the store and bought a gallon of milk
  #  Span A: the man went to the
  #  Span B: to the store and bought
  #  Span C: and bought a gallon of
  #  ...
  #
  # Now the word 'bought' will have two scores from spans B and C. We only
  # want to consider the score with "maximum context", which we define as
  # the *minimum* of its left and right context (the *sum* of left and
  # right context will always be the same, of course).
  #
  # In the example the maximum context for 'bought' would be span C since
  # it has 1 left context and 3 right context, while span B has 4 left context
  # and 0 right context.
  best_score = None
  best_span_index = None
  for (span_index, doc_span) in enumerate(doc_spans):
    end = doc_span.start + doc_span.length - 1
    if position < doc_span.start:
      continue
    if position > end:
      continue
    num_left_context = position - doc_span.start
    num_right_context = end - position
    score = min(num_left_context, num_right_context) + 0.01 * doc_span.length
    if best_score is None or score > best_score:
      best_score = score
      best_span_index = span_index

  return cur_span_index == best_span_index

In [0]:
PAR_TRAIN_FILE_1 ='gs://bert_bucket_new/bert/files/paraphrased_train_squad_1.1_full_CD.json'
par_examples_1 = read_squad_examples(
    input_file=PAR_TRAIN_FILE_1, is_training=True)

train_examples = read_squad_examples(
    input_file=TRAIN_FILE, is_training=True)

eval_examples = read_squad_examples(
    input_file=EVAL_FILE, is_training=False)

tokenizer = tokenization.FullTokenizer(
    vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)

In [0]:
train_examples_short = train_examples[:8560]
par_examples_short = par_examples_1[:8560]

In [0]:
all_examples = np.array([[ex for ex in train_examples_short+par_examples_short],
                         [ex for ex in par_examples_short+train_examples_short]])

In [0]:
#### old keep the same 

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Run BERT on SQuAD 1.1 and SQuAD 2.0."""


def _improve_answer_span(doc_tokens, input_start, input_end, tokenizer,
                         orig_answer_text):
  """Returns tokenized answer spans that better match the annotated answer."""

  # The SQuAD annotations are character based. We first project them to
  # whitespace-tokenized words. But then after WordPiece tokenization, we can
  # often find a "better match". For example:
  #
  #   Question: What year was John Smith born?
  #   Context: The leader was John Smith (1895-1943).
  #   Answer: 1895
  #
  # The original whitespace-tokenized answer will be "(1895-1943).". However
  # after tokenization, our tokens will be "( 1895 - 1943 ) .". So we can match
  # the exact answer, 1895.
  #
  # However, this is not always possible. Consider the following:
  #
  #   Question: What country is the top exporter of electornics?
  #   Context: The Japanese electronics industry is the lagest in the world.
  #   Answer: Japan
  #
  # In this case, the annotator chose "Japan" as a character sub-span of
  # the word "Japanese". Since our WordPiece tokenizer does not split
  # "Japanese", we just use "Japanese" as the annotation. This is fairly rare
  # in SQuAD, but does happen.
  tok_answer_text = " ".join(tokenizer.tokenize(orig_answer_text))

  for new_start in range(input_start, input_end + 1):
    for new_end in range(input_end, new_start - 1, -1):
      text_span = " ".join(doc_tokens[new_start:(new_end + 1)])
      if text_span == tok_answer_text:
        return (new_start, new_end)

  return (input_start, input_end)


def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 use_one_hot_embeddings):
  """Creates a classification model."""
  model = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)

  final_hidden = model.get_sequence_output()

  final_hidden_shape = modeling.get_shape_list(final_hidden, expected_rank=3)
  batch_size = final_hidden_shape[0]
  seq_length = final_hidden_shape[1]
  hidden_size = final_hidden_shape[2]

  output_weights = tf.get_variable(
      "cls/squad/output_weights", [2, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "cls/squad/output_bias", [2], initializer=tf.zeros_initializer())

  final_hidden_matrix = tf.reshape(final_hidden,
                                   [batch_size * seq_length, hidden_size])
  logits = tf.matmul(final_hidden_matrix, output_weights, transpose_b=True)
  logits = tf.nn.bias_add(logits, output_bias)

  logits = tf.reshape(logits, [batch_size, seq_length, 2])
  logits = tf.transpose(logits, [2, 0, 1])

  unstacked_logits = tf.unstack(logits, axis=0)

  (start_logits, end_logits) = (unstacked_logits[0], unstacked_logits[1])

  return (start_logits, end_logits)


def model_fn_builder(bert_config, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    unique_ids = features["unique_ids"]
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (start_logits, end_logits) = create_model(
        bert_config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)

    tvars = tf.trainable_variables()

    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      seq_length = modeling.get_shape_list(input_ids)[1]

      def compute_loss(logits, positions):
        one_hot_positions = tf.one_hot(
            positions, depth=seq_length, dtype=tf.float32)
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        loss = -tf.reduce_mean(
            tf.reduce_sum(one_hot_positions * log_probs, axis=-1))
        return loss

      start_positions = features["start_positions"]
      end_positions = features["end_positions"]

      start_loss = compute_loss(start_logits, start_positions)
      end_loss = compute_loss(end_logits, end_positions)

      total_loss = (start_loss + end_loss) / 2.0

      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)
    elif mode == tf.estimator.ModeKeys.PREDICT:
      predictions = {
          "unique_ids": unique_ids,
          "start_logits": start_logits,
          "end_logits": end_logits,
      }
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
    else:
      raise ValueError(
          "Only TRAIN and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn


In [0]:
#### new ... 

def kl(log_p, log_q):
  neg_ent = tf.reduce_sum(tf.exp(log_p) * log_p, axis=-1)
  neg_cross_ent = tf.reduce_sum(tf.exp(log_p) * log_q, axis=-1)

  kl = neg_ent - neg_cross_ent
  return kl

def js(log_p, log_q):
  log_m = 0.5 * (log_p + log_q)
  
  js = 0.5*kl(log_p, log_m) + 0.5*kl(log_q, log_m)
  
  return js

def _improve_answer_span(doc_tokens, input_start, input_end, tokenizer,
                         orig_answer_text):
  """Returns tokenized answer spans that better match the annotated answer."""

  # The SQuAD annotations are character based. We first project them to
  # whitespace-tokenized words. But then after WordPiece tokenization, we can
  # often find a "better match". For example:
  #
  #   Question: What year was John Smith born?
  #   Context: The leader was John Smith (1895-1943).
  #   Answer: 1895
  #
  # The original whitespace-tokenized answer will be "(1895-1943).". However
  # after tokenization, our tokens will be "( 1895 - 1943 ) .". So we can match
  # the exact answer, 1895.
  #
  # However, this is not always possible. Consider the following:
  #
  #   Question: What country is the top exporter of electornics?
  #   Context: The Japanese electronics industry is the lagest in the world.
  #   Answer: Japan
  #
  # In this case, the annotator chose "Japan" as a character sub-span of
  # the word "Japanese". Since our WordPiece tokenizer does not split
  # "Japanese", we just use "Japanese" as the annotation. This is fairly rare
  # in SQuAD, but does happen.
  tok_answer_text = " ".join(tokenizer.tokenize(orig_answer_text))

  for new_start in range(input_start, input_end + 1):
    for new_end in range(input_end, new_start - 1, -1):
      text_span = " ".join(doc_tokens[new_start:(new_end + 1)])
      if text_span == tok_answer_text:
        return (new_start, new_end)

  return (input_start, input_end)


def create_model(bert_config, is_training, input_ids, input_mask, segment_ids,
                 use_one_hot_embeddings):
  """Creates a classification model."""
  hidden = modeling.BertModel(
      config=bert_config,
      is_training=is_training,
      input_ids=input_ids,
      input_mask=input_mask,
      token_type_ids=segment_ids,
      use_one_hot_embeddings=use_one_hot_embeddings)
 
  final_hidden = hidden.get_sequence_output()
  final_hidden_shape = modeling.get_shape_list(final_hidden)#, expected_rank=3)
  batch_size = final_hidden_shape[0]
  seq_length = final_hidden_shape[1]
  hidden_size = final_hidden_shape[-1]
  
  print(batch_size) # 3 - > K * 3
  print(seq_length) # 384 -> K x 384 
  print(hidden_size) # 1024 
  
  with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
    output_weights = tf.get_variable(
        "cls/squad/output_weights", [2, hidden_size],
        initializer=tf.truncated_normal_initializer(stddev=0.02))

  with tf.variable_scope(tf.get_variable_scope(), reuse=tf.AUTO_REUSE):
    output_bias = tf.get_variable(
        "cls/squad/output_bias", [2], initializer=tf.zeros_initializer())
  
  final_hidden_matrix = tf.reshape(final_hidden,
                                   [batch_size * seq_length, hidden_size])
  logits = tf.matmul(final_hidden_matrix, output_weights, transpose_b=True)
  all_logits = tf.nn.bias_add(logits, output_bias)
  
  return all_logits, batch_size # return all logits for one example 

def model_fn_builder(bert_config, init_checkpoint, learning_rate,
                     num_train_steps, num_warmup_steps, use_tpu,
                     use_one_hot_embeddings):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    unique_ids = features["unique_ids"]
    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    
    unique_ids_1 = features["unique_ids_1"]
    input_ids_1 = features["input_ids_1"]
    input_mask_1 = features["input_mask_1"]
    segment_ids_1 = features["segment_ids_1"]

    input_ids = tf.concat([
          features["input_ids"],
          features["input_ids_1"]], 0)
    
    input_mask = tf.concat([
          features["input_mask"],
          features["input_mask_1"]], 0)
    
    segment_ids = tf.concat([
          features["segment_ids"],
          features["segment_ids_1"]], 0)
    
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    all_logits, batch_size = create_model(
        bert_config=bert_config,
        is_training=is_training,
        input_ids=input_ids,
        input_mask=input_mask,
        segment_ids=segment_ids,
        use_one_hot_embeddings=use_one_hot_embeddings)
    
    tvars = tf.trainable_variables()

    initialized_variable_names = {}
    scaffold_fn = None
    if init_checkpoint:
      (assignment_map, initialized_variable_names
      ) = modeling.get_assignment_map_from_checkpoint(tvars, init_checkpoint)
      if use_tpu:

        def tpu_scaffold():
          tf.train.init_from_checkpoint(init_checkpoint, assignment_map)
          return tf.train.Scaffold()

        scaffold_fn = tpu_scaffold
      else:
        tf.train.init_from_checkpoint(init_checkpoint, assignment_map)

    tf.logging.info("**** Trainable Variables ****")
    for var in tvars:
      init_string = ""
      if var.name in initialized_variable_names:
        init_string = ", *INIT_FROM_CKPT*"
      tf.logging.info("  name = %s, shape = %s%s", var.name, var.shape,
                      init_string)

    seq = 386
    
    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      seq_length = modeling.get_shape_list(input_ids)[1]

      def compute_loss(logits, positions):
        one_hot_positions = tf.one_hot(
            positions, depth=seq, dtype=tf.float32)
        
        log_probs = tf.nn.log_softmax(logits, axis=-1)
        loss = -tf.reduce_mean(
            tf.reduce_sum(one_hot_positions * log_probs, axis=-1))
        return loss

      first_logits = all_logits[:int(batch_size/2)*seq]
      first_logits = tf.reshape(first_logits, [int(batch_size/2), seq, 2])
      first_logits = tf.transpose(first_logits, [2, 0, 1])

      unstacked_logits = tf.unstack(first_logits, axis=0)

      (start_logits, end_logits) = (unstacked_logits[0], unstacked_logits[1])

      start_positions = features["start_positions"]
      end_positions = features["end_positions"]

      start_loss = compute_loss(start_logits, start_positions) 
      end_loss = compute_loss(end_logits, end_positions)

      sup_loss = (start_loss + end_loss) / 2.0 # supervised loss per one example 

      print("PROCESSED SUPERVISZED LOSS")

      # unsupervised loss 

      if ("unique_ids_1") in features:
      
        # separate logits  
        first_logits = all_logits[:int(batch_size/2)*seq_length]
        second_logits = all_logits[int(batch_size/2)*seq_length:2*int(batch_size/2)*seq_length]
        
        # for each get start and end logits  
        first_logits = tf.reshape(first_logits, [int(batch_size/2), seq, 2])
        first_logits = tf.transpose(first_logits, [2, 0, 1])

        unstacked_logits_0 = tf.unstack(first_logits, axis=0)

        (start_logits_0, end_logits_0) = (unstacked_logits_0[0], unstacked_logits_0[1]) 

        second_logits = tf.reshape(second_logits, [int(batch_size/2), seq, 2])
        second_logits = tf.transpose(second_logits, [2, 0, 1])

        unstacked_logits_1 = tf.unstack(second_logits, axis=0)

        (start_logits_1, end_logits_1) = (unstacked_logits_1[0], unstacked_logits_1[1]) 

        
        # convert to probabilities 
        start_prob_0 = tf.nn.log_softmax(start_logits_0, axis=-1)
        end_prob_0 = tf.nn.log_softmax(end_logits_0, axis=-1)
        start_prob_1 = tf.nn.log_softmax(start_logits_1, axis=-1)
        end_prob_1 = tf.nn.log_softmax(end_logits_1, axis=-1)
        
        # KL divergence / summetrified KL divergence
        unsup_loss = tf.reduce_mean(js(start_prob_0, start_prob_1))# + tf.reduce_mean(kl(start_prob_1, start_prob_0))) / 2.0
        
        alpha = 0.5
        
        print("PROCESSED UNSUP LOSS")
    
        # total loss 
        total_loss = alpha*(sup_loss) + (1-alpha)*unsup_loss
         
      else:
        
        total_loss = sup_loss
     
      train_op = optimization.create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)
      
      print("PROCESSED TRAIN OP")
      
  
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op,
          scaffold_fn=scaffold_fn)
      
      print("OUTPUT_SPEC is done")
      
    elif mode == tf.estimator.ModeKeys.PREDICT:
      
      predictions = {
          "unique_ids": unique_ids,
          "start_logits": start_logits,
          "end_logits": end_logits,
      }
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, predictions=predictions, scaffold_fn=scaffold_fn)
    else:
      raise ValueError(
          "Only TRAIN and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn

In [0]:
def input_fn_builder(input_file, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""
  
  name_to_features = {
      "unique_ids": tf.FixedLenFeature([], tf.int64),
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      
      "unique_ids_1": tf.FixedLenFeature([], tf.int64),
      "input_ids_1": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask_1": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids_1": tf.FixedLenFeature([seq_length], tf.int64),
      
  }
  
  if is_training:
    name_to_features["start_positions"] = tf.FixedLenFeature([], tf.int64)
    name_to_features["end_positions"] = tf.FixedLenFeature([], tf.int64)
  
    name_to_features["start_positions_1"] = tf.FixedLenFeature([], tf.int64)
    name_to_features["end_positions_1"] = tf.FixedLenFeature([], tf.int64)
    
 
  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)
    print("Example: " + str(example))
    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.to_int32(t)
      example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    batch_size = params["batch_size"]

    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)
    d.repeat()
    #d = StreamingFilesDataset(input_file)
    
    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))
    
    return d
  
  return input_fn

In [0]:
RawResult = collections.namedtuple("RawResult",
                                   ["unique_id", "start_logits", "end_logits"])


def write_predictions(all_examples, all_features, all_results, n_best_size,
                      max_answer_length, do_lower_case, output_prediction_file,
                      output_nbest_file, output_null_log_odds_file):
  """Write final predictions to the json file and log-odds of null if needed."""
  tf.logging.info("Writing predictions to: %s" % (output_prediction_file))
  tf.logging.info("Writing nbest to: %s" % (output_nbest_file))

  example_index_to_features = collections.defaultdict(list)
  for feature in all_features:
    example_index_to_features[feature.example_index].append(feature)

  unique_id_to_result = {}
  for result in all_results:
    unique_id_to_result[result.unique_id] = result

  _PrelimPrediction = collections.namedtuple(  # pylint: disable=invalid-name
      "PrelimPrediction",
      ["feature_index", "start_index", "end_index", "start_logit", "end_logit"])

  all_predictions = collections.OrderedDict()
  all_nbest_json = collections.OrderedDict()
  scores_diff_json = collections.OrderedDict()

  for (example_index, example) in enumerate(all_examples):
    features = example_index_to_features[example_index]

    prelim_predictions = []
    # keep track of the minimum score of null start+end of position 0
    score_null = 1000000  # large and positive
    min_null_feature_index = 0  # the paragraph slice with min mull score
    null_start_logit = 0  # the start logit at the slice with min null score
    null_end_logit = 0  # the end logit at the slice with min null score
    for (feature_index, feature) in enumerate(features):
      result = unique_id_to_result[feature.unique_id]
      start_indexes = _get_best_indexes(result.start_logits, n_best_size)
      end_indexes = _get_best_indexes(result.end_logits, n_best_size)
      # if we could have irrelevant answers, get the min score of irrelevant
      if WITH_NEGATIVE:
        feature_null_score = result.start_logits[0] + result.end_logits[0]
        if feature_null_score < score_null:
          score_null = feature_null_score
          min_null_feature_index = feature_index
          null_start_logit = result.start_logits[0]
          null_end_logit = result.end_logits[0]
      for start_index in start_indexes:
        for end_index in end_indexes:
          # We could hypothetically create invalid predictions, e.g., predict
          # that the start of the span is in the question. We throw out all
          # invalid predictions.
          if start_index >= len(feature.tokens):
            continue
          if end_index >= len(feature.tokens):
            continue
          if start_index not in feature.token_to_orig_map:
            continue
          if end_index not in feature.token_to_orig_map:
            continue
          if not feature.token_is_max_context.get(start_index, False):
            continue
          if end_index < start_index:
            continue
          length = end_index - start_index + 1
          if length > max_answer_length:
            continue
          prelim_predictions.append(
              _PrelimPrediction(
                  feature_index=feature_index,
                  start_index=start_index,
                  end_index=end_index,
                  start_logit=result.start_logits[start_index],
                  end_logit=result.end_logits[end_index]))

    if WITH_NEGATIVE:
      prelim_predictions.append(
          _PrelimPrediction(
              feature_index=min_null_feature_index,
              start_index=0,
              end_index=0,
              start_logit=null_start_logit,
              end_logit=null_end_logit))
    prelim_predictions = sorted(
        prelim_predictions,
        key=lambda x: (x.start_logit + x.end_logit),
        reverse=True)

    _NbestPrediction = collections.namedtuple(  # pylint: disable=invalid-name
        "NbestPrediction", ["text", "start_logit", "end_logit"])

    seen_predictions = {}
    nbest = []
    for pred in prelim_predictions:
      if len(nbest) >= n_best_size:
        break
      feature = features[pred.feature_index]
      if pred.start_index > 0:  # this is a non-null prediction
        tok_tokens = feature.tokens[pred.start_index:(pred.end_index + 1)]
        orig_doc_start = feature.token_to_orig_map[pred.start_index]
        orig_doc_end = feature.token_to_orig_map[pred.end_index]
        orig_tokens = example.doc_tokens[orig_doc_start:(orig_doc_end + 1)]
        tok_text = " ".join(tok_tokens)

        # De-tokenize WordPieces that have been split off.
        tok_text = tok_text.replace(" ##", "")
        tok_text = tok_text.replace("##", "")

        # Clean whitespace
        tok_text = tok_text.strip()
        tok_text = " ".join(tok_text.split())
        orig_text = " ".join(orig_tokens)

        final_text = get_final_text(tok_text, orig_text, do_lower_case)
        if final_text in seen_predictions:
          continue

        seen_predictions[final_text] = True
      else:
        final_text = ""
        seen_predictions[final_text] = True

      nbest.append(
          _NbestPrediction(
              text=final_text,
              start_logit=pred.start_logit,
              end_logit=pred.end_logit))

    # if we didn't inlude the empty option in the n-best, inlcude it
    if WITH_NEGATIVE:
      if "" not in seen_predictions:
        nbest.append(
            _NbestPrediction(
                text="", start_logit=null_start_logit,
                end_logit=null_end_logit))
    # In very rare edge cases we could have no valid predictions. So we
    # just create a nonce prediction in this case to avoid failure.
    if not nbest:
      nbest.append(
          _NbestPrediction(text="empty", start_logit=0.0, end_logit=0.0))

    assert len(nbest) >= 1

    total_scores = []
    best_non_null_entry = None
    for entry in nbest:
      total_scores.append(entry.start_logit + entry.end_logit)
      if not best_non_null_entry:
        if entry.text:
          best_non_null_entry = entry

    probs = _compute_softmax(total_scores)

    nbest_json = []
    for (i, entry) in enumerate(nbest):
      output = collections.OrderedDict()
      output["text"] = entry.text
      output["probability"] = probs[i]
      output["start_logit"] = entry.start_logit
      output["end_logit"] = entry.end_logit
      nbest_json.append(output)

    assert len(nbest_json) >= 1

    if not WITH_NEGATIVE:
      all_predictions[example.qas_id] = nbest_json[0]["text"]
    else:
      # predict "" iff the null score - the score of best non-null > threshold
      score_diff = score_null - best_non_null_entry.start_logit - (
          best_non_null_entry.end_logit)
      scores_diff_json[example.qas_id] = score_diff
      if score_diff > NULL_SCORE_DIFF_THRESHOLD:
        all_predictions[example.qas_id] = ""
      else:
        all_predictions[example.qas_id] = best_non_null_entry.text

    all_nbest_json[example.qas_id] = nbest_json
    
  with tf.gfile.GFile(output_prediction_file, "w") as writer:
    writer.write(json.dumps(all_predictions, indent=4) + "\n")

  with tf.gfile.GFile(output_nbest_file, "w") as writer:
    writer.write(json.dumps(all_nbest_json, indent=4) + "\n")

  if WITH_NEGATIVE:
    with tf.gfile.GFile(output_null_log_odds_file, "w") as writer:
      writer.write(json.dumps(scores_diff_json, indent=4) + "\n")


def get_final_text(pred_text, orig_text, do_lower_case):
  """Project the tokenized prediction back to the original text."""

  # When we created the data, we kept track of the alignment between original
  # (whitespace tokenized) tokens and our WordPiece tokenized tokens. So
  # now `orig_text` contains the span of our original text corresponding to the
  # span that we predicted.
  #
  # However, `orig_text` may contain extra characters that we don't want in
  # our prediction.
  #
  # For example, let's say:
  #   pred_text = steve smith
  #   orig_text = Steve Smith's
  #
  # We don't want to return `orig_text` because it contains the extra "'s".
  #
  # We don't want to return `pred_text` because it's already been normalized
  # (the SQuAD eval script also does punctuation stripping/lower casing but
  # our tokenizer does additional normalization like stripping accent
  # characters).
  #
  # What we really want to return is "Steve Smith".
  #
  # Therefore, we have to apply a semi-complicated alignment heruistic between
  # `pred_text` and `orig_text` to get a character-to-charcter alignment. This
  # can fail in certain cases in which case we just return `orig_text`.

  def _strip_spaces(text):
    ns_chars = []
    ns_to_s_map = collections.OrderedDict()
    for (i, c) in enumerate(text):
      if c == " ":
        continue
      ns_to_s_map[len(ns_chars)] = i
      ns_chars.append(c)
    ns_text = "".join(ns_chars)
    return (ns_text, ns_to_s_map)

  # We first tokenize `orig_text`, strip whitespace from the result
  # and `pred_text`, and check if they are the same length. If they are
  # NOT the same length, the heuristic has failed. If they are the same
  # length, we assume the characters are one-to-one aligned.
  tokenizer = tokenization.BasicTokenizer(do_lower_case=do_lower_case)

  tok_text = " ".join(tokenizer.tokenize(orig_text))

  start_position = tok_text.find(pred_text)
  if start_position == -1:
    if VERBOSE_LOGGING:
      tf.logging.info(
          "Unable to find text: '%s' in '%s'" % (pred_text, orig_text))
    return orig_text
  end_position = start_position + len(pred_text) - 1

  (orig_ns_text, orig_ns_to_s_map) = _strip_spaces(orig_text)
  (tok_ns_text, tok_ns_to_s_map) = _strip_spaces(tok_text)

  if len(orig_ns_text) != len(tok_ns_text):
    if VERBOSE_LOGGING:
      tf.logging.info("Length not equal after stripping spaces: '%s' vs '%s'",
                      orig_ns_text, tok_ns_text)
    return orig_text

  # We then project the characters in `pred_text` back to `orig_text` using
  # the character-to-character alignment.
  tok_s_to_ns_map = {}
  for (i, tok_index) in six.iteritems(tok_ns_to_s_map):
    tok_s_to_ns_map[tok_index] = i

  orig_start_position = None
  if start_position in tok_s_to_ns_map:
    ns_start_position = tok_s_to_ns_map[start_position]
    if ns_start_position in orig_ns_to_s_map:
      orig_start_position = orig_ns_to_s_map[ns_start_position]

  if orig_start_position is None:
    if VERBOSE_LOGGING:
      tf.logging.info("Couldn't map start position")
    return orig_text

  orig_end_position = None
  if end_position in tok_s_to_ns_map:
    ns_end_position = tok_s_to_ns_map[end_position]
    if ns_end_position in orig_ns_to_s_map:
      orig_end_position = orig_ns_to_s_map[ns_end_position]

  if orig_end_position is None:
    if VERBOSE_LOGGING:
      tf.logging.info("Couldn't map end position")
    return orig_text

  output_text = orig_text[orig_start_position:(orig_end_position + 1)]
  return output_text


def _get_best_indexes(logits, n_best_size):
  """Get the n-best logits from a list."""
  index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)

  best_indexes = []
  for i in range(len(index_and_score)):
    if i >= n_best_size:
      break
    best_indexes.append(index_and_score[i][0])
  return best_indexes


def _compute_softmax(scores):
  """Compute softmax probability over raw logits."""
  if not scores:
    return []

  max_score = None
  for score in scores:
    if max_score is None or score > max_score:
      max_score = score

  exp_scores = []
  total_sum = 0.0
  for score in scores:
    x = math.exp(score - max_score)
    exp_scores.append(x)
    total_sum += x

  probs = []
  for score in exp_scores:
    probs.append(score / total_sum)
  return probs


class FeatureWriter(object):
  """Writes InputFeature to TF example file."""

  def __init__(self, filename, is_training):
    self.filename = filename
    self.is_training = is_training
    self.num_features = 0
    self._writer = tf.python_io.TFRecordWriter(filename)

  def process_feature(self, feature_matrix):
    """Write a InputFeature to the TFRecordWriter as a tf.train.Example."""
    self.num_features += 1

    def create_int_feature(values):
      feature = tf.train.Feature(
          int64_list=tf.train.Int64List(value=list(values)))
      return feature
 
    # all features for each example 
 
    features = collections.OrderedDict()
    if self.is_training:
      
      features["unique_ids"] = create_int_feature([feature_matrix[0].unique_id])
      features["input_ids"] = create_int_feature(feature_matrix[0].input_ids)
      features["input_mask"] = create_int_feature(feature_matrix[0].input_mask)
      features["segment_ids"] = create_int_feature(feature_matrix[0].segment_ids)

      features["unique_ids_1"] = create_int_feature([feature_matrix[1].unique_id])
      features["input_ids_1"] = create_int_feature(feature_matrix[1].input_ids)
      features["input_mask_1"] = create_int_feature(feature_matrix[1].input_mask)
      features["segment_ids_1"] = create_int_feature(feature_matrix[1].segment_ids)

      features["start_positions"] = create_int_feature([feature_matrix[0].start_position])
      features["end_positions"] = create_int_feature([feature_matrix[0].end_position])
      
      features["start_positions_1"] = create_int_feature([feature_matrix[1].start_position])
      features["end_positions_1"] = create_int_feature([feature_matrix[1].end_position])
      
      impossible = 0
      if feature_matrix[0].is_impossible:
        impossible = 1
      features["is_impossible"] = create_int_feature([impossible])

    else:  
      
      features["unique_ids"] = create_int_feature([feature_matrix.unique_id])
      features["input_ids"] = create_int_feature(feature_matrix.input_ids)
      features["input_mask"] = create_int_feature(feature_matrix.input_mask)
      features["segment_ids"] = create_int_feature(feature_matrix.segment_ids)
      
    tf_example = tf.train.Example(features=tf.train.Features(feature=features))
    self._writer.write(tf_example.SerializeToString())  
      
  def close(self):
    self._writer.close()

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)

tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
    tf_random_seed=1,
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

In [0]:
def get_estimator(train_batch_size, lr, train_epochs, warmup_proportion, num_train_steps):
  num_warmup_steps = int(num_train_steps * warmup_proportion)

  model_fn = model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=INIT_CHECKPOINT,
      learning_rate=lr,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=True,
      use_one_hot_embeddings=True)

  # If TPU is not available, this will fall back to normal Estimator on CPU
  # or GPU.
  estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=True,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=train_batch_size,
      predict_batch_size=EVAL_BATCH_SIZE)
  return estimator

In [0]:
def get_train_input_fn(examples):
    train_writer = FeatureWriter(
      filename=os.path.join(OUTPUT_DIR, "train.tf_record"),
      is_training=True)
    
    all_tokens_1 = convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=MAX_SEQ_LENGTH,
        doc_stride=DOC_STRIDE,
        max_query_length=64,
        is_training=True,
        output_fn=train_writer.process_feature)

    train_writer.close()

    train_input_fn = input_fn_builder(
        input_file = train_writer.filename,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=True)

    return train_input_fn

In [0]:
def get_eval_input_fn(examples):
    eval_writer = FeatureWriter(
        filename=os.path.join(OUTPUT_DIR, "eval.tf_record"),
        is_training=False)
    eval_features = []

    def append_feature(feature):
      eval_features.append(feature)
      eval_writer.process_feature(feature)

    all_tokens= convert_examples_to_features_eval(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=MAX_SEQ_LENGTH,
        doc_stride=DOC_STRIDE,
        max_query_length=64,
        is_training=False,
        output_fn=append_feature)
    
    eval_writer.close()

    predict_input_fn = input_fn_builder(
        input_file=eval_writer.filename,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=True)
    
    return predict_input_fn, eval_features, all_tokens

In [0]:
NUM_TRAIN_EPOCHS = 1
num_train_steps = int(len(train_examples_short) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
estimator = get_estimator(TRAIN_BATCH_SIZE, LEARNING_RATE, NUM_TRAIN_EPOCHS, WARMUP_PROPORTION, num_train_steps)

train_input_fn = get_train_input_fn(all_examples)


W0905 15:31:53.110416 140003207100288 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f54ab9768c8>) includes params argument, but params are not passed to Estimator.
I0905 15:31:53.113046 140003207100288 estimator.py:209] Using config: {'_model_dir': 'gs://bert_bucket_new/bert/models/SQuAD_1.1_large_new_1', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.94.95.10:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.Clu

34812
34812
Processed All Features!!!!!


In [0]:
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

I0905 15:36:08.828463 140003207100288 tpu_system_metadata.py:78] Querying Tensorflow master (grpc://10.94.95.10:8470) for TPU system metadata.
I0905 15:36:08.842790 140003207100288 tpu_system_metadata.py:148] Found TPU system:
I0905 15:36:08.845844 140003207100288 tpu_system_metadata.py:149] *** Num TPU Cores: 8
I0905 15:36:08.846923 140003207100288 tpu_system_metadata.py:150] *** Num TPU Workers: 1
I0905 15:36:08.847861 140003207100288 tpu_system_metadata.py:152] *** Num TPU Cores Per Worker: 8
I0905 15:36:08.849061 140003207100288 tpu_system_metadata.py:154] *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 6396140126797673023)
I0905 15:36:08.849969 140003207100288 tpu_system_metadata.py:154] *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 15519625609306575865)
I0905 15:36:08.850873 140003207100288 tpu_system_metadata.py:154] *** Available Device: _DeviceAttributes(/job:worker/replica:0/tas

Example: {'end_positions': <tf.Tensor 'ParseSingleExample/ParseSingleExample:0' shape=() dtype=int64>, 'end_positions_1': <tf.Tensor 'ParseSingleExample/ParseSingleExample:1' shape=() dtype=int64>, 'input_ids': <tf.Tensor 'ParseSingleExample/ParseSingleExample:2' shape=(386,) dtype=int64>, 'input_mask': <tf.Tensor 'ParseSingleExample/ParseSingleExample:3' shape=(386,) dtype=int64>, 'segment_ids': <tf.Tensor 'ParseSingleExample/ParseSingleExample:4' shape=(386,) dtype=int64>, 'start_positions': <tf.Tensor 'ParseSingleExample/ParseSingleExample:5' shape=() dtype=int64>, 'start_positions_1': <tf.Tensor 'ParseSingleExample/ParseSingleExample:6' shape=() dtype=int64>, 'unique_ids': <tf.Tensor 'ParseSingleExample/ParseSingleExample:7' shape=() dtype=int64>}


I0905 15:36:16.343756 140003207100288 <ipython-input-81-d426c81925ed>:125] **** Trainable Variables ****
I0905 15:36:16.347114 140003207100288 <ipython-input-81-d426c81925ed>:131]   name = bert/embeddings/word_embeddings:0, shape = (30522, 1024), *INIT_FROM_CKPT*
I0905 15:36:16.348604 140003207100288 <ipython-input-81-d426c81925ed>:131]   name = bert/embeddings/token_type_embeddings:0, shape = (2, 1024), *INIT_FROM_CKPT*
I0905 15:36:16.349848 140003207100288 <ipython-input-81-d426c81925ed>:131]   name = bert/embeddings/position_embeddings:0, shape = (512, 1024), *INIT_FROM_CKPT*
I0905 15:36:16.351167 140003207100288 <ipython-input-81-d426c81925ed>:131]   name = bert/embeddings/LayerNorm/beta:0, shape = (1024,), *INIT_FROM_CKPT*
I0905 15:36:16.352478 140003207100288 <ipython-input-81-d426c81925ed>:131]   name = bert/embeddings/LayerNorm/gamma:0, shape = (1024,), *INIT_FROM_CKPT*
I0905 15:36:16.353683 140003207100288 <ipython-input-81-d426c81925ed>:131]   name = bert/encoder/layer_0/atte

In [0]:
PARAPHRASED_FILE ='gs://bert_bucket_new/bert/files/dev-v1.1_paraphrased.json'
eval_examples_CD_small = read_squad_examples(
    input_file=PARAPHRASED_FILE, is_training=False)

In [0]:
EVAL_FILE ='gs://bert_bucket_new/bert/files/adversarial.json'
eval_examples_adv = read_squad_examples(
    input_file=EVAL_FILE, is_training=False)

In [0]:
estimator = get_estimator(TRAIN_BATCH_SIZE, LEARNING_RATE, NUM_TRAIN_EPOCHS, WARMUP_PROPORTION, num_train_steps)
eval_input_fn, eval_features, all_tokens = get_eval_input_fn(eval_examples_adv)

W0905 15:48:27.192561 140003207100288 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f5495ae5ea0>) includes params argument, but params are not passed to Estimator.
I0905 15:48:27.195663 140003207100288 estimator.py:209] Using config: {'_model_dir': 'gs://bert_bucket_new/bert/models/SQuAD_1.1_large_new_1', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.94.95.10:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.Clu

In [0]:
all_results_p = []
for result in estimator.predict(
      eval_input_fn, yield_single_examples=True):
    if len(all_results_p) % 100 == 0:
      tf.logging.info("Processing example: %d" % (len(all_results_p)))
    unique_id = int(result["unique_ids"])
    start_logits = [float(x) for x in result["start_logits"].flat]
    end_logits = [float(x) for x in result["end_logits"].flat]
    all_results_p.append(
        RawResult(
            unique_id=unique_id,
            start_logits=start_logits,
            end_logits=end_logits))

output_prediction_file = os.path.join(OUTPUT_DIR, "predictions_adv.json")
output_nbest_file = os.path.join(OUTPUT_DIR, "nbest_predictions_adv.json")
output_null_log_odds_file = os.path.join(OUTPUT_DIR, "null_odds.json")

write_predictions(eval_examples_adv, eval_features, all_results_p,
                   20, 64,
                   DO_LOWER_CASE, output_prediction_file,
                   output_nbest_file, output_null_log_odds_file)

I0905 15:48:42.839162 140003207100288 tpu_system_metadata.py:78] Querying Tensorflow master (grpc://10.94.95.10:8470) for TPU system metadata.
I0905 15:48:42.852412 140003207100288 tpu_system_metadata.py:148] Found TPU system:
I0905 15:48:42.853356 140003207100288 tpu_system_metadata.py:149] *** Num TPU Cores: 8
I0905 15:48:42.854421 140003207100288 tpu_system_metadata.py:150] *** Num TPU Workers: 1
I0905 15:48:42.855395 140003207100288 tpu_system_metadata.py:152] *** Num TPU Cores Per Worker: 8
I0905 15:48:42.856418 140003207100288 tpu_system_metadata.py:154] *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 6396140126797673023)
I0905 15:48:42.857424 140003207100288 tpu_system_metadata.py:154] *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 15519625609306575865)
I0905 15:48:42.858537 140003207100288 tpu_system_metadata.py:154] *** Available Device: _DeviceAttributes(/job:worker/replica:0/tas

Example: {'input_ids': <tf.Tensor 'ParseSingleExample/ParseSingleExample:0' shape=(386,) dtype=int64>, 'input_mask': <tf.Tensor 'ParseSingleExample/ParseSingleExample:1' shape=(386,) dtype=int64>, 'segment_ids': <tf.Tensor 'ParseSingleExample/ParseSingleExample:2' shape=(386,) dtype=int64>, 'unique_ids': <tf.Tensor 'ParseSingleExample/ParseSingleExample:3' shape=() dtype=int64>}


I0905 15:48:48.109251 140003207100288 <ipython-input-97-d426c81925ed>:125] **** Trainable Variables ****
I0905 15:48:48.110487 140003207100288 <ipython-input-97-d426c81925ed>:131]   name = bert/embeddings/word_embeddings:0, shape = (30522, 1024), *INIT_FROM_CKPT*
I0905 15:48:48.111582 140003207100288 <ipython-input-97-d426c81925ed>:131]   name = bert/embeddings/token_type_embeddings:0, shape = (2, 1024), *INIT_FROM_CKPT*
I0905 15:48:48.112775 140003207100288 <ipython-input-97-d426c81925ed>:131]   name = bert/embeddings/position_embeddings:0, shape = (512, 1024), *INIT_FROM_CKPT*
I0905 15:48:48.113907 140003207100288 <ipython-input-97-d426c81925ed>:131]   name = bert/embeddings/LayerNorm/beta:0, shape = (1024,), *INIT_FROM_CKPT*
I0905 15:48:48.115129 140003207100288 <ipython-input-97-d426c81925ed>:131]   name = bert/embeddings/LayerNorm/gamma:0, shape = (1024,), *INIT_FROM_CKPT*
I0905 15:48:48.116284 140003207100288 <ipython-input-97-d426c81925ed>:131]   name = bert/encoder/layer_0/atte

In [0]:
def convert_examples_to_features_eval(examples, tokenizer, max_seq_length,
                                 doc_stride, max_query_length, is_training,
                                 output_fn):
  """Loads a data file into a list of `InputBatch`s."""

  all_tokens = []
  unique_id = 1000000000

  for (example_index, example) in enumerate(examples):
    query_tokens = tokenizer.tokenize(example.question_text)

    if len(query_tokens) > max_query_length:
      query_tokens = query_tokens[0:max_query_length]

    tok_to_orig_index = []
    orig_to_tok_index = []
    all_doc_tokens = []
    for (i, token) in enumerate(example.doc_tokens):
      orig_to_tok_index.append(len(all_doc_tokens))
      sub_tokens = tokenizer.tokenize(token)
      for sub_token in sub_tokens:
        tok_to_orig_index.append(i)
        all_doc_tokens.append(sub_token)

    tok_start_position = None
    tok_end_position = None
    if is_training and example.is_impossible:
      tok_start_position = -1
      tok_end_position = -1
    if is_training and not example.is_impossible:
      tok_start_position = orig_to_tok_index[example.start_position]
      if example.end_position < len(example.doc_tokens) - 1:
        tok_end_position = orig_to_tok_index[example.end_position + 1] - 1
      else:
        tok_end_position = len(all_doc_tokens) - 1
      (tok_start_position, tok_end_position) = _improve_answer_span(
          all_doc_tokens, tok_start_position, tok_end_position, tokenizer,
          example.orig_answer_text)

    # The -3 accounts for [CLS], [SEP] and [SEP]
    max_tokens_for_doc = max_seq_length - len(query_tokens) - 3

    # We can have documents that are longer than the maximum sequence length.
    # To deal with this we do a sliding window approach, where we take chunks
    # of the up to our max length with a stride of `doc_stride`.
    _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
        "DocSpan", ["start", "length"])
    doc_spans = []
    start_offset = 0
    while start_offset < len(all_doc_tokens):
      length = len(all_doc_tokens) - start_offset
      if length > max_tokens_for_doc:
        length = max_tokens_for_doc
      doc_spans.append(_DocSpan(start=start_offset, length=length))
      if start_offset + length == len(all_doc_tokens):
        break
      start_offset += min(length, doc_stride)

    for (doc_span_index, doc_span) in enumerate(doc_spans):
      tokens = []
      token_to_orig_map = {}
      token_is_max_context = {}
      segment_ids = []
      tokens.append("[CLS]")
      segment_ids.append(0)
      for token in query_tokens:
        tokens.append(token)
        segment_ids.append(0)
      tokens.append("[SEP]")
      segment_ids.append(0)

      for i in range(doc_span.length):
        split_token_index = doc_span.start + i
        token_to_orig_map[len(tokens)] = tok_to_orig_index[split_token_index]

        is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                               split_token_index)
        token_is_max_context[len(tokens)] = is_max_context
        tokens.append(all_doc_tokens[split_token_index])
        segment_ids.append(1)
      tokens.append("[SEP]")
      segment_ids.append(1)

      input_ids = tokenizer.convert_tokens_to_ids(tokens)

      # The mask has 1 for real tokens and 0 for padding tokens. Only real
      # tokens are attended to.
      input_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

      assert len(input_ids) == max_seq_length
      assert len(input_mask) == max_seq_length
      assert len(segment_ids) == max_seq_length 

      start_position = None
      end_position = None
      if is_training and not example.is_impossible:
        # For training, if our document chunk does not contain an annotation
        # we throw it out, since there is nothing to predict.
        doc_start = doc_span.start
        doc_end = doc_span.start + doc_span.length - 1
        out_of_span = False
        if not (tok_start_position >= doc_start and
                tok_end_position <= doc_end):
          out_of_span = True
        if out_of_span:
          start_position = 0
          end_position = 0
        else:
          doc_offset = len(query_tokens) + 2
          start_position = tok_start_position - doc_start + doc_offset
          end_position = tok_end_position - doc_start + doc_offset

      if is_training and example.is_impossible:
        start_position = 0
        end_position = 0

      if example_index < 20:
        tf.logging.info("*** Example ***")
        tf.logging.info("unique_id: %s" % (unique_id))
        tf.logging.info("example_index: %s" % (example_index))
        tf.logging.info("doc_span_index: %s" % (doc_span_index))
        tf.logging.info("tokens: %s" % " ".join(
            [tokenization.printable_text(x) for x in tokens]))
        tf.logging.info("token_to_orig_map: %s" % " ".join(
            ["%d:%d" % (x, y) for (x, y) in six.iteritems(token_to_orig_map)]))
        tf.logging.info("token_is_max_context: %s" % " ".join([
            "%d:%s" % (x, y) for (x, y) in six.iteritems(token_is_max_context)
        ]))
        tf.logging.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
        tf.logging.info(
            "input_mask: %s" % " ".join([str(x) for x in input_mask]))
        tf.logging.info(
            "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
        if is_training and example.is_impossible:
          tf.logging.info("impossible example")
        if is_training and not example.is_impossible:
          answer_text = " ".join(tokens[start_position:(end_position + 1)])
          tf.logging.info("start_position: %d" % (start_position))
          tf.logging.info("end_position: %d" % (end_position))
          tf.logging.info(
              "answer: %s" % (tokenization.printable_text(answer_text)))

      feature = InputFeatures(
          unique_id=unique_id,
          example_index=example_index,
          doc_span_index=doc_span_index,
          tokens=tokens,
          token_to_orig_map=token_to_orig_map,
          token_is_max_context=token_is_max_context,
          input_ids=input_ids,
          input_mask=input_mask,
          segment_ids=segment_ids,
          start_position=start_position,
          end_position=end_position,
          is_impossible=example.is_impossible)

      # Run callback
      output_fn(feature)     
    unique_id += 1
    all_tokens.append(tokens)

  return all_tokens

In [0]:
def get_logits(all_results, all_tokens): 
  start_logits = []
  end_logits = []
  context_tokens = []
  for i in range(len(all_tokens)):
    index_to_remove = all_tokens[i].index('[SEP]')+1
    context_tokens.append(all_tokens[i][index_to_remove+1:-1])

    start_logits.append(all_results[i].start_logits[index_to_remove:index_to_remove+len(context_tokens[i])])
    end_logits.append(all_results[i].end_logits[index_to_remove:index_to_remove+len(context_tokens[i])])

  assert len(context_tokens[i]) == len(start_logits[i])
  assert len(context_tokens[i]) == len(end_logits[i])
  
  return start_logits, end_logits

In [0]:
def softmax(x):
  return np.exp(x)/np.sum(np.exp(x))

def kl(p,q):
  return np.sum(np.where(p != 0, p*np.log(p/q), 0))

def prob_dist(start_logits, end_logits):
  start_probs = []
  for start_s in start_logits:
    start_probs.append(softmax(start_s))

  end_probs = []
  for end_s in end_logits:
    end_probs.append(softmax(end_s))
    
  return start_probs, end_probs

In [0]:
start_logits_p, end_logits_p = get_logits(all_results_p, all_tokens)
start_logits_q, end_logits_q = get_logits(all_results_q, all_tokens)

In [0]:
start_probs_p, end_probs_p = prob_dist(start_logits_p, end_logits_p)
start_probs_q, end_probs_q = prob_dist(start_logits_q, end_logits_q)

In [0]:
# Symmetric KL, JS, Wassertstein distance 
kl_total = []
EMD_total = []
JS = []

for i in range(len(probs_p)):
  probs_p[i] = np.array(probs_p[i])
  probs_q[i] = np.array(probs_q[i])
  
  kl_total.append((kl(probs_p[i], probs_q[i]) + kl(probs_q[i], probs_p[i]))/2)
  EMD_total.append(wasserstein_distance(probs_p[i], probs_q[i]))
  
  JS.append(distance.jensenshannon(probs_p[i], probs_q[i]))

In [0]:
print(np.mean(EMD_total))
print(np.mean(kl_total))
print(np.mean(JS))

In [0]:
PARAPHRASED_FILE ='gs://bert_bucket_new/bert/files/dev-v1.1_paraphrased.json'
eval_examples = read_squad_examples(
    input_file=PARAPHRASED_FILE, is_training=False)


In [0]:
eval_input_fn, eval_features, all_tokens = get_eval_input_fn(eval_examples)

In [0]:
all_results_q = []
for result in estimator.predict(
      eval_input_fn, yield_single_examples=True):
    if len(all_results_q) % 100 == 0:
      tf.logging.info("Processing example: %d" % (len(all_results_q)))
    unique_id = int(result["unique_ids"])
    start_logits = [float(x) for x in result["start_logits"].flat]
    end_logits = [float(x) for x in result["end_logits"].flat]
    all_results_q.append(
        RawResult(
            unique_id=unique_id,
            start_logits=start_logits,
            end_logits=end_logits))

output_prediction_file = os.path.join(OUTPUT_DIR, "predictions_KL_0.2.json")
output_nbest_file = os.path.join(OUTPUT_DIR, "nbest_predictions_KL_0.2.json")
output_null_log_odds_file = os.path.join(OUTPUT_DIR, "null_odds_KL_0.2.json")

write_predictions(eval_examples, eval_features, all_results_q,
                   20, 64,
                   DO_LOWER_CASE, output_prediction_file,
                   output_nbest_file, output_null_log_odds_file)